In [ ]:
pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 24.6 MB/s eta 0:00:00


In [ ]:
pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from haversine import haversine
from sklearn.preprocessing import StandardScaler
import numpy as np
import math
import dataclasses
import folium

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cafe_data= pd.read_excel('/content/drive/MyDrive/project_코스추천/data/자카드유사도 카페.xlsx')
cafe = pd.read_excel('/content/drive/MyDrive/project_코스추천/data/cafe.xlsx')
restaurant = pd.read_excel('/content/drive/MyDrive/project_코스추천/data/restaurant.xlsx')
alcohol = pd.read_excel('/content/drive/MyDrive/project_코스추천/data/alcohol.xlsx')
alcohol = alcohol.drop(['Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1'], axis=1)
alcohol = alcohol.drop(index=20)
dist_rate = pd.read_excel('/content/drive/MyDrive/project_코스추천/data/dist_rate.xlsx')
cafe_data=cafe_data[['mood', 'cafe_name', 'jac_sim']]
cafe_dist=pd.read_excel('/content/drive/MyDrive/project_코스추천/data/카페 좌표.xlsx')


In [ ]:
# 결과창
print("""
*********언어 기반 한남동 데이트 코스 추천*********

[오늘의 분위기를 선택해 주세요!]

1. 한적한 곳에서 여유를 즐기고 싶은 하루 🌿

2. 맛있는 것만 잔뜩 먹고 싶은 하루 🍽️

3. 인스타 감성, 힙한 곳에서 인생샷 남기고 싶은 하루 📷

4. 알록달록, 아기자기한 곳에서 기분 전환 하고 싶은 하루 🌈

5. 진부한 것 말고, 색다른 분위기를 경험하고 싶은 하루 💎

6. 연인과 로맨틱한 데이트를 하고 싶은 하루 💝

""")



*********언어 기반 한남동 데이트 코스 추천*********

[오늘의 분위기를 선택해 주세요!]

1. 한적한 곳에서 여유를 즐기고 싶은 하루 🌿

2. 맛있는 것만 잔뜩 먹고 싶은 하루 🍽️

3. 인스타 감성, 힙한 곳에서 인생샷 남기고 싶은 하루 📷

4. 알록달록, 아기자기한 곳에서 기분 전환 하고 싶은 하루 🌈

5. 진부한 것 말고, 색다른 분위기를 경험하고 싶은 하루 💎

6. 연인과 로맨틱한 데이트를 하고 싶은 하루 💝




In [ ]:
choice_1= input('번호를 입력해 주세요:')

번호를 입력해 주세요:5


In [ ]:
choice_1 = int(choice_1)
choice_df = cafe_data[cafe_data.mood == choice_1]
choice_df = choice_df.reset_index(drop=True)


if choice_1 == 1:
  choice_1 = '한적한 곳에서 여유를 즐기고 싶은 하루 🌿'
elif choice_1 == 2:
  choice_1 = '맛있는 것만 잔뜩 먹고 싶은 하루 🍽️'
elif choice_1 == 3:
  choice_1 = '인스타 감성, 힙한 곳에서 인생샷 남기고 싶은 하루 📷'
elif choice_1 == 4:
  choice_1 = '알록달록, 아기자기한 곳에서 기분 전환 하고 싶은 하루 🌈'
elif choice_1 == 5:
  choice_1 = '진부한 것 말고, 색다른 분위기를 경험하고 싶은 하루 💎'
elif choice_1 == 6:
  choice_1 = '연인과 로맨틱한 데이트를 하고 싶은 하루 💝'


print("""
이런 카페는 어떠세요? 🤔
""")

for i in range(0,5):
    num=i+1
    cafe_name= choice_df['cafe_name'][i][4:]
    print(f'{num}. {cafe_name}')


이런 카페는 어떠세요? 🤔 

1. 오아시스한남점
2. KONTEMPORARY1
3. 제이엘디저트바
4. 아이라이크마운틴
5. 보난자커피


In [ ]:
choice_2 = input('카페를 선택해주세요: ')

카페를 선택해주세요: 보난자커피


In [ ]:
cafe = cafe[cafe['place'] == choice_2]
train_data = pd.concat([cafe, restaurant])
train_data = train_data.reset_index(drop=True)

In [ ]:
corpus = []
for words in train_data['cleaned']:
  corpus.append(words)


# 사전 훈련된 Word2Vec을 다운받아서 이를 초기 단어 벡터값으로 워드 임베딩
from gensim.models import Word2Vec
word2vec_model = Word2Vec(corpus, size=100, window=5, min_count=5, workers=4, sg=0)


# 단어벡터의 평균 구하기
def vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line:
            if word in word2vec_model.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = word2vec_model[word]
                else:
                    doc2vec = doc2vec + word2vec_model[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list


document_embedding_list = vectors(corpus)

# 각 벡터간 코사인 유사도
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

<ipython-input-17-1317c69997bf>:24: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc2vec = word2vec_model[word]
<ipython-input-17-1317c69997bf>:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc2vec = doc2vec + word2vec_model[word]


코사인 유사도 매트릭스의 크기 : (138, 138)


In [ ]:
def recommenatdions(place):
    places = train_data[['place']]

    # 장소를 입력하면 해당 장소의 인덱스를 리턴받아 idx에 저장.
    indices = pd.Series(train_data.index, index = train_data['place']).drop_duplicates()
    idx = indices[place]

    # 입력된 장소과 리뷰(document embedding)가 유사한 장소 5개 선정.
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]

    # 가장 유사한 장소 5개의 인덱스
    place_indices = [i[0] for i in sim_scores]

    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.
    recommend = places.iloc[place_indices].reset_index(drop=True)

    #print(recommend)
    return recommend

In [ ]:
print('식당은 이곳을 추천드려요! 😋')
recommend = recommenatdions(choice_2)
result = pd.DataFrame(recommend)
result.loc[5] = [choice_2]

data = pd.merge(result, dist_rate, left_on='place', right_on='place', how='inner')

vector = []
for i in range(len(data)):
    vect = float(data['lon'][i]), float(data['lat'][i])
    vector.append(vect)
data['vect'] = vector

dist=[]
for i in range(len(data)):
    for k in range(len(data)):
        if i < k:
            try:
                distance = 1 / haversine(data['vect'][i], data['vect'][k], unit='km')
            except ZeroDivisionError:
                0
    dist.append(distance)


ssum = []
for i in range(5):
  distt = 1/(haversine(data['vect'][5], data['vect'][i], unit='km'))
  zz = ((distt-4.548656830828885) / 7.539707931416238) + ((data['rate'][i]-4.436814159292034) / 0.16390027988510905)
  ssum.append(zz)

score = pd.DataFrame({'place': data['place'][0:5], 'score':ssum})
score = score.sort_values('score', ascending=False)

for i in range(0,5):                                                                                                                        #
    num=i+1
    place_name= score['place'][i]
    print(f'{num}. {place_name}')

식당은 이곳을 추천드려요! 😋
1. 토담집
2. 도노
3. 리틀넥
4. 세컨드키친사운즈
5. 파이프그라운드


In [ ]:
choice_3 = input('식당을 선택해주세요:')

식당을 선택해주세요:세컨드키친사운즈


In [ ]:
restaurant = restaurant[restaurant['place'] == choice_3]
train_data = pd.concat([restaurant, alcohol])
train_data = train_data.reset_index(drop=True)


In [ ]:
corpus = []
for words in train_data['cleaned']:
  corpus.append(words)


# 사전 훈련된 Word2Vec을 다운받아서 이를 초기 단어 벡터값으로 워드 임베딩
from gensim.models import Word2Vec
word2vec_model = Word2Vec(corpus, size=100, window=5, min_count=5, workers=4, sg=0)


document_embedding_list = vectors(corpus)

# 각 벡터간 코사인 유사도
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)

<ipython-input-17-1317c69997bf>:24: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc2vec = word2vec_model[word]
<ipython-input-17-1317c69997bf>:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc2vec = doc2vec + word2vec_model[word]


In [ ]:
def recommenatdions(place):
    places = train_data[['place']]

    # 장소를 입력하면 해당 장소의 인덱스를 리턴받아 idx에 저장.
    indices = pd.Series(train_data.index, index = train_data['place']).drop_duplicates()
    idx = indices[place]

    # 입력된 장소과 리뷰(document embedding)가 유사한 장소 5개 선정.
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]

    # 가장 유사한 장소 5개의 인덱스
    place_indices = [i[0] for i in sim_scores]

    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.
    recommend = places.iloc[place_indices].reset_index(drop=True)

    #print(recommend)
    return recommend

In [ ]:
print('술 한 잔까지 하신다면? 💃')
recommend = recommenatdions(choice_3)
result = pd.DataFrame(recommend)
result.loc[5] = [choice_3]

data = pd.merge(result, dist_rate, left_on='place', right_on='place', how='inner')

vector = []
for i in range(len(data)):
    vect = float(data['lon'][i]), float(data['lat'][i])
    vector.append(vect)
data['vect'] = vector

dist=[]
for i in range(len(data)):
    for k in range(len(data)):
        if i < k:
            try:
                distance = 1 / haversine(data['vect'][i], data['vect'][k], unit='km')
            except ZeroDivisionError:
                0
    dist.append(distance)

ssum = []
for i in range(5):
  distt = 1/(haversine(data['vect'][5], data['vect'][i], unit='km'))
  zz = ((distt-4.548656830828885) / 7.539707931416238) + ((data['rate'][i]-4.436814159292034) / 0.16390027988510905)
  ssum.append(zz)

score = pd.DataFrame({'place': data['place'][0:5], 'score':ssum})
score = score.sort_values('score', ascending=False)

for i in range(0,5):                                                                                                                        #<--- 수정부분
    num=i+1
    place_name= score['place'][i]
    print(f'{num}. {place_name}')

술 한 잔까지 하신다면? 💃
1. 쿱한남
2. 홉가든한남
3. 심퍼티쿠시
4. 로프트230
5. 뮤땅


In [ ]:
choice_4 = input('술집을 선택해주세요:')

술집을 선택해주세요:홉가든한남


In [ ]:
print(f'''
********* 최종 코스 제안 *********
{choice_1}, 여기에서 만들어 보시는 건 어떨까요?
''')
print(choice_2)
print(choice_3)
print(choice_4)


********* 최종 코스 제안 *********
진부한 것 말고, 색다른 분위기를 경험하고 싶은 하루 💎, 여기에서 만들어 보시는 건 어떨까요?

보난자커피
세컨드키친사운즈
홉가든한남


In [ ]:
# 지도 기본 위치 choice_2 카페로 지정
target_cafe=cafe_dist[cafe_dist.name == choice_2]
target_cafe= target_cafe.drop_duplicates(['name'], keep='first')
target_cafe= target_cafe.reset_index()
target_cafe_dist=[target_cafe['lon'][0], target_cafe['lat'][0]]

In [ ]:
dist_rate

,Unnamed: 0,place,lon,lat,rate
0,0.0,월당,37.536399,127.000807,4.43
1,1.0,옛이야기,37.534337,127.008370,4.58
2,2.0,고다이,37.534601,127.008774,4.75
3,3.0,라멘81번옥,37.530122,127.006502,4.31
4,4.0,라멘81번옥,37.537494,127.001358,4.31
...,...,...,...,...,...
222,222.0,레베르,37.536684,127.001437,4.43
223,223.0,바스왈로,37.536875,126.999522,4.43
224,224.0,히어로도넛,37.532175,126.999890,4.43
225,225.0,다원골뱅이,37.532522,127.005848,4.24


In [ ]:
# 레스토랑 좌표 생성
target_rest= dist_rate[dist_rate.place == choice_3]
target_rest= target_rest.drop_duplicates(['place'], keep='first')
target_rest= target_rest.reset_index()
target_rest_dist=[target_rest['lon'][0], target_rest['lat'][0]]

# 술집 좌표 생성
target_al= dist_rate[dist_rate.place == choice_4]
target_al= target_al.drop_duplicates(['place'], keep='first')
target_al= target_al.reset_index()
target_al_dist=[target_al['lon'][0], target_al['lat'][0]]

In [ ]:
# 지도 생성
m = folium.Map(
    location = target_cafe_dist,
    zoom_start = 17
)

# 지도에 마커 찍기
marker1= folium.Marker(
    location = target_cafe_dist,
    tooltip= choice_2,
    icon = folium.Icon(
    	color = 'blue',
      icon = 'coffee',
      prefix='fa')
).add_to(m)

marker2=folium.Marker(
    location = target_rest_dist,
    tooltip= choice_3,
    icon = folium.Icon(
    	color = 'red',
      icon = 'cutlery',
      prefix= 'glyphicon')
).add_to(m)

marker3=folium.Marker(
    location = target_al_dist,
    tooltip= choice_4,
    icon = folium.Icon(
    	color = 'black',
      icon = 'beer',
      prefix='fa')
).add_to(m)

marker1.add_to(m)
marker2.add_to(m)
marker3.add_to(m)

m # 지도 시각화